In [2]:
import os
import tensorflow as tf
import numpy as np
from rnn_model.attention_lstm import bilstm_with_attention
from tensorflow.contrib import rnn

In [3]:
num_class=30
embedding_size = 256
NUM_CLASS = 20
BATCH_SIZE = 64
NUM_EPOCHS = 1
WORD_MAX_LEN = 80
vocabulary_size = len(word_dict)
from tensorflow.contrib import rnn
       
      
num_class=30
embedding_size = 256
num_hidden = 256
num_layers = 2
learning_rate = 1e-3
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [None, 80], name="x")
x_len = tf.reduce_sum(tf.sign(x), 1)
y = tf.placeholder(tf.int32, [None], name="y")
is_training = tf.placeholder(tf.bool, [], name="is_training")
global_step = tf.Variable(0, trainable=False)
keep_prob = tf.where(is_training, 0.5, 1.0)

with tf.name_scope("embedding"):
    init_embeddings = tf.random_uniform([vocabulary_size, embedding_size])
    embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
    x_emb = tf.nn.embedding_lookup(embeddings, x)

with tf.name_scope("birnn"):
    fw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    bw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    fw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in fw_cells]
    bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in bw_cells]

rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
fw_cells, bw_cells, x_emb, sequence_length=x_len, dtype=tf.float32)

with tf.name_scope("attention"):
    attention_score = tf.nn.softmax(tf.layers.dense(rnn_outputs, 1, activation=tf.nn.tanh), axis=1)
    attention_out = tf.squeeze(
    tf.matmul(tf.transpose(rnn_outputs, perm=[0, 2, 1]), attention_score),axis=-1)


with tf.name_scope("output"):
    logits = tf.layers.dense(attention_out, num_class, activation=tf.nn.softmax)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "attentionRnn.ckpt-35125"
sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)


probabilities = sess.run(tf.nn.top_k(logits,2,sorted=True), feed_dict = {x: np.array(test_x[3]).reshape(1,50),is_training:False})
print("부정: {}%".format(probabilities[0][0][0]*100),"긍정: {}%".format(probabilities[0][0][1]*100))

attention_weights = sess.run(attention_score,feed_dict = {x: np.array(test_x[3]).reshape(1,50),is_training:False})

NameError: name 'word_dict' is not defined